# Overjeno šifriranje

Demonstracija implementacije Fernet v Javi.

Cilji laboratorijske vaje so sledeči:
- Uporabi AES-GCM v Pythonu
- Uporabi AES-GCM v Javi
- Implementacija Phonebook v Pythonu
- Branje phonebook v Javi
- Implementacija Fernet v Pythonu

In [9]:
import os
import time
import base64

from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

## Naloga 1: Overjeno šifriranje z AES-GCM
Za ogrevanje implementirajmo funkciji `enc_gcm(key, pt, nonce)` in `dec_gcm(key, ct, nonce)`. Prva vrne tajnopis, druga čistopis. 

Za podrobnosti poglejte v [dokumentacijo AES-GCM.](https://cryptography.io/en/latest/hazmat/primitives/aead/#cryptography.hazmat.primitives.ciphers.aead.AESGCM)

In [2]:
def enc_gcm(key, pt, nonce):
    return AESGCM(key).encrypt(nonce, pt, None)

In [3]:
def test_enc_gcm():
    key = bytes.fromhex("cedaaa2d9f2418e79b5376971a857a1e")
    nonce = bytes.fromhex("622dbb32a4fd85a492081deeb982d805")
    message = b"Hello World!"
    ct = bytes.fromhex("bbe99dd675e28ddc8cfe5c9cb2e7039431f03c767421d049d3e52d6f")
        
    assert enc_gcm(key, message, nonce) == ct

test_enc_gcm()    

In [4]:
def dec_gcm(key, ct, nonce):
    return AESGCM(key).decrypt(nonce, ct, None)

In [6]:
def test_dec_gcm():
    key = bytes.fromhex("cedaaa2d9f2418e79b5376971a857a1e")
    nonce = bytes.fromhex("622dbb32a4fd85a492081deeb982d805")
    ct = bytes.fromhex("bbe99dd675e28ddc8cfe5c9cb2e7039431f03c767421d049d3e52d6f")
    message = b"Hello World!"
    assert dec_gcm(key, ct, nonce) == message

test_dec_gcm()    

## Naloga 2: Overjeno šifriranje z AES-GCM v Javi

Odprite Javanski projekt: mapa `java-ae`; najlažje, da z IntelliJ odprete datoteko `pom.xml` in ko vas program vpraša na kakšen način jo želite odpreti, izberete kot projekt.

Za začetek si poglejte primer v datoteki `GCMExample.java`: v celoti vas vodi skozi primer uporabe šifre AES-GCM. Bodite pozorni:
- kako ustvarimo ključ,
- kako šifro instanciiramo,
- kako jo inicializiramo,
- kdaj se nastavi IV in kako ga preberemo,
- kako IV izrecno nastavimo (npr. pri dešifriranju).

Zatem implementirajte nalogo v datoteki `CommunicationExampleGCM.java`. V agentih uporabite šifro AES-GCM in z njo zavarujte sporočilo, ki ga Ana pošlje Boru. Ključ definirajte globalno v metodi `main(String[])`.

## Naloga 3: Izpeljava ključa iz gesla

S pomočjo knjižnice cryptography implementirajte funkcijo `gen_key(password, salt, iterations)`, ki iz podanega gesla izpelje simetrični ključ s pomočjo algoritma PBKDF2.

Argument `salt` naj bo naključna vrednost dolžine 16 bajtov, argument iterations pa število iteracij: privzeta vrednost argumenta naj bo milijon. Ustvarjen ključ naj bo dolg 16 bajtov. Za zgoščevalno funkcijo uporabite SHA256.

Podrobnosti [poiščite v dokumentaciji.](https://cryptography.io/en/latest/hazmat/primitives/key-derivation-functions/#pbkdf2)

In [41]:
def gen_key(password, salt, iterations=1000000):
    return PBKDF2HMAC(
        algorithm=hashes.SHA256(), 
        length=16, 
        salt=salt, 
        iterations=iterations).derive(password.encode("utf8"))

gen_key("password1234", os.urandom(16))

b'\x0c\xaf*\xf7\x92\x1d\xb5fJ\x935\x1a\xe9w\x93\x8a'

In [42]:
def test_gen_key():
    assert gen_key("hunter2", bytes.fromhex("e24701ca2d923d43e50a405ec718f3af"), 1000).hex() == "8c0ab28934ac682f98e73e6edf498de8"
    assert gen_key("hunter2", bytes.fromhex("e24701ca2d923d43e50a405ec718f3af")).hex() == "0ffa82d350f20600bf865df0eebb5027"

test_gen_key()

## Naloga 4: Zavarujte aplikacijo Telefonski imenik

Aplikaciji Telefonski imenik v datoteki `pb.py` dodajte overjeno šifriranje, in sicer uporabite AES-GCM. Ključ za šifriranje izpeljite iz gesla. 

Podobno kot smo naredili pri 4. laboratorijski vaji, dodajte postopek šifriranja in dešifriranja zaporedoma v funkciji `save_phone_book(phone_book, file, password)` in `load_phone_book(file, password)`; obe funkciji spremenite tako, da bosta namesto argumenta `key`, vzeli argument `password`.

Iz argumenta `password` nato s pomočjo algoritma PBKDF2 izpeljite ključ. Argument password preberite takoj ob zagonu programa in ga shranite v spremenljivko, ki jo lahko nato podate kot argument pri klicu funkcij `save_phone_book(phone_book, file, password)` in `load_phone_book(file, password)`.

Pozor: tokrat boste morali v datoteko shraniti kar tri podatke: sol, IV in tajnopis, kjer slednji že vsebuje značko overitvene kode sporočila. Slednje lahko dosežete tako, da bajte, ki sestavljajo sol, IV in tajnopis konkatenirate in shranite v datoteko. Pri branju pa prebrano vsebino datoteke razpakirate v tri dele:
- prvih 16 bajtov je sol,
- drugih šestanjst bajtov je IV,
- preostali bajti so tajnopis z značko overitvene kode sporočila.


## Naloga 5: Preberite vsebino telefonskega imenika v Javi

Sedaj v Javanskem projektu dokončajte implementacijo branja datoteke `data/phonebook.bin` v datoteki `ReadPhoneBookFile.java`.

Cilj je zgolj prebrati, pravilno dešifrirati in na standardni izhod izpisati vsebino imenika tj. vsebino datoteke `data/phonebook.bin`. Ostalih funkcionalnosti aplikacije telefonskega imenika ni potrebno implementirati.

## Naloga 6: Implementacija šifre Fernet

Implementirajte funkcije overjene šifre Fernet. Knižnica cryptography to implementacijo že ponuja, pri tej nalogi pa jo boste implementirali sami po podani specifikaciji. Omenjena naloga je odličen primer, kako v celoto povežete znanje o kodiranju informacij, šifriranju in overjanju sporočil.

Viri:
- [Implementacija Fernet v cryptography](https://cryptography.io/en/latest/fernet/)
- [Specifikacija Fernet](https://github.com/fernet/spec/blob/master/Spec.md)

In [ ]:
def fernet_gen_key():
    return base64.urlsafe_b64encode(os.urandom(32))

In [ ]:
def fernet_encrypt(key, message):
    # unpack and decode keys
    k_mac, k_enc = base64.urlsafe_b64decode(key)[:16], base64.urlsafe_b64decode(key)[16:]    
    
    version = (128).to_bytes(1, byteorder='big')
    timestamp = int(time.time()).to_bytes(8, byteorder='big')
    iv = os.urandom(16)
            
    # padding
    padder = padding.PKCS7(128).padder() # CBC padding
    padded_msg = padder.update(message) + padder.finalize()
    
    # encryption
    enc = Cipher(algorithms.AES(k_enc), modes.CBC(iv)).encryptor() # AES-CBC
    ct = enc.update(padded_msg) + enc.finalize()
    
    # mac
    h = hmac.HMAC(k_mac, hashes.SHA256()) # hmac
    h.update(version + timestamp + iv + ct)
    tag = h.finalize()
    
    # token
    return base64.urlsafe_b64encode(version + timestamp + iv + ct + tag)    

In [ ]:
def fernet_decrypt(key, token, ttl=None):
    # unpack and decode keys
    k_mac, k_enc = base64.urlsafe_b64decode(key)[:16], base64.urlsafe_b64decode(key)[16:]
    
    # token
    token = base64.urlsafe_b64decode(token)
    
    version = token[0]
    assert version == 128, "Invalid version"
    
    timestamp = int.from_bytes(token[1:9], byteorder='big') 
    if ttl:
        assert int(time.time()) - timestamp > ttl, "Maximum age expired"
    
    # mac
    h = hmac.HMAC(k_mac, hashes.SHA256()) # hmac
    h.update(token[:-32])
    h.verify(token[-32:])
     
    # decryption
    iv = token[9:25]
    ct = token[25:-32]
    dec = Cipher(algorithms.AES(k_enc), modes.CBC(iv)).decryptor() # AES-CBC
    pt = dec.update(ct) + dec.finalize()
    
    # remove pad
    unpadder = padding.PKCS7(128).unpadder() # CBC padding
    msg = unpadder.update(pt) + unpadder.finalize()
    return msg

In [ ]:
key = fernet_gen_key()
ct = fernet_encrypt(key, b"Hello World! A very long message, surely more than a single block.")
fernet_decrypt(key, ct)

In [46]:
from cryptography.fernet import Fernet

In [47]:
def load_java():
    with open("../data/fernet-java.key", "rb") as h:
        key = h.read()

    with open("../data/fernet-java.ct", "rb") as h:
        ct = h.read()
        
    return key, ct

In [48]:
key, ct = load_java()
f = Fernet(key)
f.decrypt(ct).decode("utf8")

'Hello Wold! Tole je primer sporočila.'